<a href="https://colab.research.google.com/github/lanmasco/DDoS/blob/lanmasco-patch-1/CitratoDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python matplotlib torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import pandas as pd
import sys
# Import the cv2_imshow function from google.colab.patches
from google.colab.patches import cv2_imshow


# ----------- [1] Persiapan Gambar -----------

image_path = 'IMG_9243.jpg'
if not os.path.exists(image_path):
    print(f"[ERROR] File '{image_path}' tidak ditemukan.")
    sys.exit(1)

img_color = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

# Binarisasi
_, binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)

# Kernel
kernel = np.ones((5, 5), np.uint8)

# Erosi
erosion = cv2.erode(binary, kernel, iterations=1)

# Opening
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# ----------- [2] Simpan & Tampilkan Citra Pra-Pemrosesan -----------

output_dir = 'output_proses_citra'
os.makedirs(output_dir, exist_ok=True)

cv2.imwrite(f'{output_dir}/1_grayscale.jpg', img_gray)
cv2.imwrite(f'{output_dir}/2_binary.jpg', binary)
cv2.imwrite(f'{output_dir}/3_erosion.jpg', erosion)
cv2.imwrite(f'{output_dir}/4_opening.jpg', opening)

# Tampilkan ke monitor pakai Matplotlib
titles = ['Grayscale', 'Binary', 'Erosion', 'Opening']
images = [img_gray, binary, erosion, opening]

plt.figure(figsize=(10, 8))
for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.suptitle('Tahapan Pra-Pemrosesan Citra', fontsize=16)
plt.tight_layout()
plt.show()

# ----------- [3] Deteksi Mobil dengan YOLOv5 -----------

print("[INFO] Memuat YOLOv5...")
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

results = model(img_color)
df = results.pandas().xyxy[0]

cars = df[df['name'] == 'car']
if cars.empty:
    print("Tidak ada mobil terdeteksi.")
else:
    print("=== Deteksi Mobil ===")
    print(cars[['name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']])

cars.to_csv(os.path.join(output_dir, 'mobil_detected.csv'), index=False)

# Gambar bounding box ke gambar RGB
img_detect = img_rgb.copy()
for _, row in cars.iterrows():
    x1, y1 = int(row['xmin']), int(row['ymin'])
    x2, y2 = int(row['xmax']), int(row['ymax'])
    label = f"Car {row['confidence']:.2f}"
    cv2.rectangle(img_detect, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img_detect, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

cv2.imwrite(os.path.join(output_dir, 'deteksi_mobil.jpg'), cv2.cvtColor(img_detect, cv2.COLOR_RGB2BGR))

# ----------- [4] Tampilkan Hasil Deteksi ke Monitor -----------

plt.figure(figsize=(8, 6))
plt.imshow(img_detect)
plt.title('Deteksi Mobil (YOLOv5)')
plt.axis('off')
plt.tight_layout()
plt.show()

# ----------- [5] Tampilkan juga pakai OpenCV (opsional) -----------

# Use cv2_imshow instead of cv2.imshow for displaying images in Colab
cv2_imshow(img_gray)
cv2_imshow(binary)
cv2_imshow(erosion)
cv2_imshow(opening)
cv2_imshow(cv2.cvtColor(img_detect, cv2.COLOR_RGB2BGR))

print("[INFO] Tekan tombol apa pun di jendela gambar untuk keluar...")
# cv2.waitKey(0) # waitKey is also not supported in Colab
# cv2.destroyAllWindows() # destroyAllWindows is also not supported in Colab